# <font color="darkred" size="+4">**Particle Physics**</font>
Centrale-Supélec - ST4 - 2024

<font color="darkred" size="+3">**Tutorial to perform Likelihood fits with iMinuit**</font>



Tutorial from Hans Dembinski, current iMinuit maintainer.

This tutorial is very good to illustrate a simple yet powerful framework to fit parameters from likelihoods with iMinuit.

![image|100px](https://scikit-hep.org/iminuit/_images/iminuit_logo.svg)

iMinuit documentation is available there:

https://scikit-hep.org/iminuit/index.html


We first process step by step and then collect information on what is required with iMinuit to indeed fit correctly the dataset with out model.

Installing first the packages not available in Colab.

In [ ]:
!pip install iminuit
!pip install resample

Then importing the minimum setup to process this notebook

In [ ]:
from iminuit import Minuit
from iminuit.cost import ExtendedBinnedNLL
from iminuit.cost import ExtendedUnbinnedNLL
from resample import bootstrap

from scipy import stats
import numpy as np
from matplotlib import pyplot as plt

We generate some random data, with a global exponential shape background, and a gaussian signal

In [ ]:
rng = np.random.default_rng(1)  # setting the seed to get reproducible output

x = np.append(stats.norm(0.5, 0.05).rvs(size=1_000, random_state=rng),
              stats.expon(0.0, 0.5).rvs(size=10_000, random_state=rng))

Plotting now this dataset:

In [ ]:
bins = 50
n, xe = np.histogram(x, bins=bins, range=(0, 1))
xc = 0.5 * (xe[1:] + xe[:-1])  # compute bin centers
dx = xe[1:] - xe[:-1] # bin width
# plt.hist(x, bins=bins, range=(0, 1))
plt.errorbar(xc, n, yerr=np.sqrt(n), fmt='ko');

We now define the model for the bin content, to be used with binned likelihood fits.

In [ ]:
# extended binned maximum likelihood fit
# ExtendedBinnedNLL wants a model that returns the expected counts per bin
def signal(xe, ns, mu, sigma):
    return ns * stats.norm(mu, sigma).cdf(xe)

def background(xe, nb, lambd):
    return nb * stats.expon(0.0, lambd).cdf(xe)

def total(xe, ns, mu, sigma, nb, lambd):
    return signal(xe, ns, mu, sigma) + background(xe, nb, lambd)


In [ ]:
cost = ExtendedBinnedNLL(n, xe, total)

In [ ]:
m = Minuit(cost, ns=0, mu=1, sigma=1, nb=0, lambd=1)
m.migrad()
# >>> Give an invalid minimum...

In [ ]:
# Ask for more information
cost.verbose = 1
m = Minuit(cost, ns=0, mu=1, sigma=1, nb=0, lambd=1)
m.migrad()

We observe NaN's (Not-a-number) ouputs in the previous call. This means that the computations gave rise to undefinite numbers. This is typically the case when numbers are negative and only positive numbers are allowed, etc.

In [ ]:
# Setting bounds: σ>0 and λ>0
# sigma and lambd must be positive...
cost.verbose = 1
m = Minuit(cost, ns=0, mu=0, sigma=1, nb=0, lambd=10)
m.limits['lambd'] = (0, None)
m.limits['sigma'] = (0, None)
m.migrad()

In [ ]:
# >>> ran over its call limit.
# %% Increase nb. of possible calls
m = Minuit(cost, ns=0, mu=0, sigma=1, nb=0, lambd=10)
m.limits['lambd'] = (0, None)
m.limits['sigma'] = (0, None)
m.migrad(ncall=10_000)

In [ ]:
# %% Plot that solution
def plot_model(xe, cdf, **kwargs):  # helper function
    plt.plot(xe, np.append(np.diff(cdf), np.nan),
             drawstyle="steps-post", **kwargs)


plt.hist(x, bins=50, range=(0, 1))
plt.errorbar(xe[:-1]+np.mean(np.diff(xe))/2, n, yerr=np.sqrt(n),
             fmt='ko')
plot_model(xe, total(xe, *m.values[:]), linewidth=2)

In [ ]:
# %%
# signal window is roughly 0.3 to 0.7, let's mask that out

cost.mask = (xc < 0.3) | (0.7 < xc)

cost.verbose = 0  # turn verbosity off again, this fit should not cause trouble

m = Minuit(cost, ns=0, mu=1, sigma=1, nb=0, lambd=1)
m.limits['lambd'] = (0, None)
m.limits['sigma'] = (0, None)

# fix signal parameters for next fit and set signal amplitude to zero
m.fixed[:3] = True
m.values["ns"] = 0

m.migrad()

In [ ]:
# %% unmask signal region now
cost.mask = None
m.fixed[:] = False
m.fixed["nb"] = True
m.fixed["lambd"] = True
m.values["ns"] = 1
m.migrad()

In [ ]:
# %%
m.fixed[:] = False
m.migrad()

Instead of step by step as above when soliving a puzzle for minimization (which is extremly convenient by the way!) it is possible to setup from start with the following options:

In [ ]:
# %% Everything right from start with following instructions
m = Minuit(cost, ns=0, mu=0, sigma=1, nb=0, lambd=10) # Fresh new definition of Minuit state
m.limits['ns'] = (0, None)
m.limits['nb'] = (0, None)
m.limits['mu'] = (0.4, 0.6)
m.limits['lambd'] = (0, 2)
m.limits['sigma'] = (0, 0.2)
m.migrad()

If still experiencing "INVALID MINIMUM", it might be because the number of iteration steps ran over the limit. We can increase it:

In [ ]:
m = Minuit(cost, ns=0, mu=0, sigma=1, nb=0, lambd=10) # Fresh new definition of Minuit state
m.limits['ns'] = (0, None)
m.limits['nb'] = (0, None)
m.limits['mu'] = (0.4, 0.6)
m.limits['lambd'] = (0, 2)
m.limits['sigma'] = (0, 0.2)
m.migrad(ncall=10_000)

Possibility to resample the initial dataset to process bootstrap samples and estimate variations of the parameters, cheking their mean and standard deviation this way.

In [ ]:
# Bonus: resampling and bootstrap
cost.n = np.histogram(x, bins=bins, range=(0, 1))[0]
m = Minuit(cost, ns=0, mu=0, sigma=1, nb=0, lambd=10)
m.limits['ns'] = (0, None)
m.limits['nb'] = (0, None)
m.limits['mu'] = (0.4, 0.6)
m.limits['lambd'] = (0, 2)
m.limits['sigma'] = (0, 0.2)
m.migrad(ncall=10_000)
print(f"nfcn for original fit = {m.fmin.nfcn}")
m

In [ ]:
errors = m.errors[:]
nfcn = m.fmin.nfcn

In [ ]:
rng = np.random.default_rng(1)
b_value = []
b_nfcn = []
for i, b_sample in enumerate(bootstrap.resample(x, size=50,
                                                random_state=rng)):
    cost.n = np.histogram(b_sample, bins=bins, range=(0, 1))[0]
    m.migrad()
    print(f"nfcn for fit of b_sample[{i}] = {m.fmin.nfcn}")
    assert m.valid
    b_value.append(m.values[:])
    b_nfcn.append(m.fmin.nfcn)

Check number of function calls: minuit remembers state of previous fit. So it converges faster to solution on successive calls than with fresh restart...

In [ ]:
plt.axvline(nfcn, color="C0", label="original")
plt.hist(np.diff(b_nfcn), color="C1", label="bootstrapped")
plt.xlabel("number of likelihood evaluations")
plt.legend();

We can now compare fit parameters estimated uncertainties with bootstrap standard deviations.

In [ ]:
b_cov = np.cov(np.transpose(b_value)) # be careful with b_value dimensions and np.cov call
b_errors = np.diag(b_cov) ** 0.5  # getting standard deviations.

height = 0.35
i = np.arange(5)
plt.barh(i - height/2, errors, height, label="Minuit")
plt.barh(i + height/2, b_errors, height, label="bootstrap")
plt.semilogx()
plt.yticks(i, m.parameters)
plt.xlabel("uncertainty estimate")
plt.legend();

In [ ]:
# %% Extended Unbinned NLL
from iminuit.cost import ExtendedUnbinnedNLL
# x = x[x<2]

def fs(x, mu, sigma):
    return stats.norm(mu, sigma).pdf(x)

def fb(x, lambd):
    return stats.expon(0.0, lambd).pdf(x)

def f(x, ns, mu, sigma, nb, lambd):
    return ns+nb, ns*fs(x, mu, sigma) + nb*fb(x, lambd)

In [ ]:
cost = ExtendedUnbinnedNLL(x, f)

m = Minuit(cost, ns=1, mu=1, sigma=1, nb=1, lambd=1)
m.limits['ns'] = (0, None)
m.limits['nb'] = (0, None)
m.limits['mu'] = (0.4, 0.6)
m.limits['lambd'] = (0, 2)
m.limits['sigma'] = (0, 0.2)

m.migrad(ncall=10_000)

Calling MINOS algorithm for better estimate of parameter uncertainties.

In [ ]:
m.minos()